In [52]:
import torch
with open('lorem_ipsum.txt', 'r') as f:
    text = f.read()
    text = text.replace('\n', ' ')

In [98]:
# bpe tokenizer
def get_stats(ids):
    counts = {}
    for pair in zip(ids[:-1], ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    new_ids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and (ids[i], ids[i + 1]) == pair:
            new_ids.append(idx)
            i += 2
        else:
            new_ids.append(ids[i])
            i += 1
    return new_ids

def byte_pair_encoding(text, num_merges):
    tokens = text.encode('utf-8', errors='replace')
    tokens_processed = list(map(int, tokens))
    merges = {}
    for i in range(num_merges):
        stats = get_stats(tokens_processed)
        max_pair = max(stats, key=stats.get)
        tokens_processed = merge(tokens_processed, max_pair, 256 + i)
        merges[max_pair] = 256 + i
        print(f'merging pair {max_pair} into {256 + i}')
    # Save the merges to a file
    with open('merges.txt', 'w') as f:
        for pair, token_id in merges.items():
            f.write('\t'.join([str(pair[0]), str(pair[1]), str(token_id)]) + '\n')
    return tokens_processed, merges

In [ ]:
vocab_size = 280
num_of_merges = vocab_size - 256
tokens_processed, merges = byte_pair_encoding(text, num_of_merges)

vocab = {idx : bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]
# print tokens with largest idx
def decode(ids):
    tokens = b"".join(vocab[idx] for idx in ids)
    new_text = tokens.decode('utf-8', errors='replace')
    return new_text

In [100]:
def encode(text):
    tokens = list(text.encode('utf-8'))    
    if len(tokens) >= 2:
        for pair, idx in merges.items():
            if len(tokens) < 2:
                break
            tokens = merge(tokens, pair, idx)   
    return tokens

In [101]:
print(decode(encode('This is bpe tokenizer test')))

This is bpe tokenizer test
